# Lab 10-1 Deep learning prediction using time series data
## Simple RNN 

<table class="tfo-notebook-buttons" align="left">
   <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/text_classification_rnn.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import plot_model

# Basic RNNs

### Generate the Dataset

In [ ]:
def generate_time_series(batch_size, n_steps):        
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    
    time = np.linspace(0, 1, n_steps)    
    series = 0.5* np.sin((time - offsets1)  * (freq1 * 10 + 10)) 
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2    
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # + noise    
    return series[..., np.newaxis].astype(np.float32)


In [ ]:
np.random.seed(42)

n_steps = 50
ndata = 2000
series = generate_time_series(ndata, n_steps + 1)

n1 = int(ndata*0.7)
n2 = int(ndata*0.9)

X_train, y_train = series[:n1, :n_steps], series[:n1, -1]
X_valid, y_valid = series[n1:n2, :n_steps], series[n1:n2, -1]
X_test, y_test = series[n2:, :n_steps], series[n2:, -1]

In [ ]:
X_train.shape, y_train.shape, X_valid.shape

In [ ]:
def plot_series(series, y=None, y_pred=None, x_label="$t$", y_label="$x(t)$"):
    plt.plot(series, ".-")
    if y is not None:
        plt.plot(n_steps, y, "bx", markersize=10)
    if y_pred is not None:
        plt.plot(n_steps, y_pred, "ro")
    plt.grid(True)
    if x_label:
        plt.xlabel(x_label, fontsize=16)
    if y_label:
        plt.ylabel(y_label, fontsize=16, rotation=0)
    plt.hlines(0, 0, 100, linewidth=1)
    plt.axis([0, n_steps + 1, -1, 1])

In [ ]:
plot_series(X_valid[0, :, 0], y_valid[0, 0])
plt.show()

In [ ]:
y_pred = X_valid[:, -1]
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

In [ ]:
plot_series(X_valid[0, :, 0], y_valid[0, 0], y_pred[0, 0])
plt.show()

### Computing Some Baselines

Naive predictions (just predict the last observed value):

In [ ]:
y_pred = X_valid[:, -1]
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

- 아래 그림을 잘 그리기 위해서 'n_steps'을 plot_series에 추가 해주었다.  

In [ ]:
plot_series(X_valid[0, :, 0], y_valid[0, 0], y_pred[0, 0])
plt.show()

## a1. 선형 예측 

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model_flatten = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])

model=model_flatten

model.compile(loss="mse", optimizer="adam")

In [ ]:
plot_model(model, to_file='../log/fig10_a1_Flatten.png', show_shapes=True)

In [ ]:
model.summary()

In [ ]:
%%time
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

In [ ]:
model.evaluate(X_valid, y_valid)

In [ ]:
def plot_learning_curves(loss, val_loss):
    plt.plot(np.arange(len(loss)) + 0.5, loss, "b.-", label="Training loss")
    plt.plot(np.arange(len(val_loss)) + 1, val_loss, "r.-", label="Validation loss")
    plt.gca().xaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
    #plt.axis([1, 20, 0, 0.05])
    plt.legend(fontsize=14)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.grid(True)

In [ ]:
plot_learning_curves(history.history["loss"], history.history["val_loss"])
plt.show()

In [ ]:
y_pred = model.predict(X_valid)
plot_series( X_valid[0, :, 0], y_valid[0, 0], y_pred[0, 0])
plt.show()

In [ ]:
a1 = history.history['loss'][np.argmin(history.history['loss'])]
print('The loss of basics', a1)

## a2. Simple RNN

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

simpleRNN = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

model = simpleRNN
optimizer = keras.optimizers.Adam(lr=0.005)
model.compile(loss="mse", optimizer=optimizer)

In [ ]:
plot_model(model, to_file='../log/fig10_a2_simpleRNN.png', show_shapes=True)

In [ ]:
%%time
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

In [ ]:
model.evaluate(X_valid, y_valid)

In [ ]:
plot_learning_curves(history.history["loss"], history.history["val_loss"])
plt.show()

In [ ]:
y_pred = model.predict(X_valid)
plot_series(X_valid[0, :, 0], y_valid[0, 0], y_pred[0, 0])
plt.show()

In [ ]:
a2 = history.history['loss'][np.argmin(history.history['loss'])]
print('The loss of SimpleRNN model', a2)

## a3. Deep RNNs

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

deepRNN = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

model = deepRNN
model.compile(loss="mse", optimizer="adam")

In [ ]:
plot_model(model, to_file='../log/fig10_a3_deepRNN.png', show_shapes=True)

In [ ]:
%%time
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

In [ ]:
model.evaluate(X_valid, y_valid)

In [ ]:
plot_learning_curves(history.history["loss"], history.history["val_loss"])
plt.show()

In [ ]:
y_pred = model.predict(X_valid)
plot_series(X_valid[0, :, 0], y_valid[0, 0], y_pred[0, 0])
plt.show()

In [ ]:
a3 = history.history['loss'][np.argmin(history.history['loss'])]
print('The loss of deepRNN model', a3)

## a4. Deep RNN with layer return only the last output:

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

deepRNNDense = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

model = deepRNNDense

In [ ]:
plot_model(model, to_file='../log/fig10_a4_deepRNNDense.png', show_shapes=True)

In [ ]:
model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

In [ ]:
model.evaluate(X_valid, y_valid)

In [ ]:
plot_learning_curves(history.history["loss"], history.history["val_loss"])
plt.show()

In [ ]:
y_pred = model.predict(X_valid)
plot_series(X_valid[0, :, 0], y_valid[0, 0], y_pred[0, 0])
plt.show()

In [ ]:
a4 = history.history['loss'][np.argmin(history.history['loss'])]
print('The loss of DeepRNNDense model', a4)

# Results Summary of many-to-one 

In [ ]:
models = pd.DataFrame({
    'Model': ['Flatten',  'SimpleRNN','DeepRNN','DeepRNNDense'],
    'Score': [a1, a2, a3, a4]})
models.sort_values(by='Score', ascending=True)